In [1]:
! pip install azure-storage-blob requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 28.5 MB/s eta 0:00:0000:01


In [1]:
import os
import json
import hashlib
from datetime import datetime, timedelta
from tqdm import tqdm
import requests as req
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, explode, col, array, lit, transform
from azure.storage.blob import BlobServiceClient
from hdfs import InsecureClient as HdfsClient
from io import BytesIO
from PIL import Image
from azure.storage.filedatalake import DataLakeServiceClient
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [3]:
connection_string = os.getenv('AZURE_CONNECTION_STRING')
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
spark = SparkSession.builder \
    .appName("BDMProject") \
    .master("spark://localhost:7077")\
    .getOrCreate()
spark.sparkContext.setLogLevel("OFF")

25/05/15 23:46:26 WARN Utils: Your hostname, Medio-Metro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.35 instead (on interface en0)
25/05/15 23:46:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/15 23:46:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Transportation

Find out more in: https://openrouteservice.org/dev/#/api-docs/optimization/post
Alternatives: https://github.com/graphhopper/graphhopper/blob/master/README.md#Map-Matching
https://github.com/VROOM-Project/vroom/blob/master/docs/API.md

In [4]:
from hdfs import InsecureClient

def test_spark_connection() -> bool:
    """
    Verifies the Spark session is active by performing a simple operation.
    Returns True if the session is responsive, False otherwise.
    """
    try:
        # Simple operation: create a small DataFrame and collect
        count = spark.range(0, 1).count()
        print (count == 1)
    except Exception:
        return print(False)

def test_hdfs_connection():
    try:
        hdfs_url = os.environ['HDFS_URL']
        client = InsecureClient(hdfs_url, user='hdfs')  # use appropriate user

        # List root directory
        print("Listing root directory:")
        print(client.list('/'))

        # Write a test file
        test_path = '/tmp/test_hdfs_connection.txt'
        test_content = 'HDFS connection successful!'
        client.write(test_path, data=test_content, overwrite=True)
        print(f"File written to {test_path}")

        # Read back the file
        with client.read(test_path, encoding='utf-8') as reader:
            content = reader.read()
            print("File content:")
            print(content)

        return True
    except Exception as e:
        print(f"❌ HDFS connection failed: {e}")
        return False

# Run test
if __name__ == '__main__':
    test_hdfs_connection()
    test_spark_connection()

Listing root directory:
['tmp']
File written to /tmp/test_hdfs_connection.txt
File content:
HDFS connection successful!


True


# Helper Functions

In [89]:
def build_struct(flat_schema: dict) -> StructType:
    fields = []
    for name, typ in flat_schema.items():
        if isinstance(typ, list):
            elem = typ[0]
            if isinstance(elem, dict):
                struct = build_struct(elem)
                fields.append(StructField(name, ArrayType(struct), True))
            else:
                spark_type = {'int': IntegerType(), 'float': DoubleType(), 'str': StringType(), 'bool': BooleanType()}.get(elem, StringType())
                fields.append(StructField(name, ArrayType(spark_type), True))
        else:
            spark_type = {'int': IntegerType(), 'float': DoubleType(), 'str': StringType(), 'bool': BooleanType()}.get(typ, StringType())
            fields.append(StructField(name, spark_type, True))
    return StructType(fields)

def string_to_sha256(text: str) -> str:
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

def flatten_df(df):
    from pyspark.sql.types import StructType
    flat_cols = []
    nested_cols = []
    for field in df.schema.fields:
        if isinstance(field.dataType, StructType):
            nested_cols.append(field.name)
        else:
            flat_cols.append(col(field.name))
    for nested in nested_cols:
        for f in df.schema[nested].dataType.fields:
            flat_cols.append(col(f"{nested}.{f.name}").alias(f"{nested}_{f.name}"))
    return df.select(flat_cols)

# Configuración del Data Lake
# Asume que 'file_system_name' es el filesystem de Delta Lake
def get_data_lake_service(account_url: str, credential) -> DataLakeServiceClient:
    return DataLakeServiceClient(account_url=account_url, credential=credential)

@udf(StringType())
def sha256_udf(url: str) -> str:
    return hashlib.sha256(url.encode('utf-8')).hexdigest() if url else None

@udf(StringType())
def standardized_hours_udf(ts: str) -> str:
    return datetime.fromisoformat(ts).strftime('%H:%M') if ts else None


def file_exists(fs_client, path: str) -> bool:
    try:
        file_client = fs_client.get_file_client(path)
        return file_client.exists()
    except Exception:
        return False


def upload_file(fs_client, path: str, data: bytes, overwrite=True) -> None:
    file_client = fs_client.get_file_client(path)
    if overwrite and file_client.exists():
        file_client.delete_file()
    file_client.create_file()
    file_client.append_data(data, offset=0)
    file_client.flush_data(len(data))

# ---------------------------------------------------------------------
# FLATTEN, CAST, SCHEMA Y COMPRESIÓN DE IMÁGENES (sin cambios)
# ---------------------------------------------------------------------


def load_json_schema(path: str) -> dict:
    with open(path) as f:
        return json.load(f)

def compress_image(image_bytes: bytes, max_width: int = 1024, quality: int = 75) -> bytes:
    with Image.open(BytesIO(image_bytes)) as img:
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
        if img.width > max_width:
            ratio = max_width / float(img.width)
            new_height = int(img.height * ratio)
            img = img.resize((max_width, new_height), Image.LANCZOS)
        buffer = BytesIO()
        img.save(buffer, format="JPEG", quality=quality, optimize=True)
        return buffer.getvalue()

def process_accommodation_record(record: dict, schema: dict) -> dict:
    # Crear DF inferido
    accommodation_schema_dict = json.loads(open('accommodation_schema.json').read())
    accommodation_struct = build_struct(accommodation_schema_dict)
     # Crear DF completo e inferido
    df = spark.read.json(spark.sparkContext.parallelize([json.dumps(record)]))

    # Extraer columnas planas y hasta 3 niveles de anidación

    all_cols = []
    for field in df.schema.fields:
        if isinstance(field.dataType, StructType):
            for subfield in field.dataType.fields:
                if isinstance(subfield.dataType, StructType):
                    for subsubfield in subfield.dataType.fields:
                        if isinstance(subsubfield.dataType, StructType):
                            for subsubsubfield in subsubfield.dataType.fields:
                                all_cols.append(
                                    col(f"{field.name}.{subfield.name}.{subsubfield.name}.{subsubsubfield.name}").alias(f"{field.name}_{subfield.name}_{subsubfield.name}_{subsubsubfield.name}")
                                )
                        else:
                            all_cols.append(
                                col(f"{field.name}.{subfield.name}.{subsubfield.name}").alias(f"{field.name}_{subfield.name}_{subsubfield.name}")
                            )
                else:
                    all_cols.append(
                        col(f"{field.name}.{subfield.name}").alias(f"{field.name}_{subfield.name}")
                    )
        else:
            all_cols.append(col(field.name))

    df_flat = df.select(*all_cols)

    # Generar hashes de fotos en Python
    photo_urls = record.get("property", {}).get("photoUrls", [])
    photo_hashes = [string_to_sha256(u) for u in photo_urls]
    df_final = df_flat.withColumn("property_photoHash", lit(photo_hashes))

    # Serializar y validar con esquema oficial
    json_flat = df_final.toJSON().first()
    df_valid = spark.read.schema(accommodation_struct).json(spark.sparkContext.parallelize([json_flat]))
    return df_valid.collect()[0].asDict()

# --- Procesar registro de clima con Spark flatten y luego aplicar esquema ---
def process_weather_record(raw: dict, schema: dict) -> dict:
    weather_schema_dict = json.loads(open('weather_schema.json').read())
    weather_struct = build_struct(weather_schema_dict)
    raw_hourly = raw['hourly']
    raw_hourly['timestamp'] = raw_hourly['time']
    raw_hourly['time'] = [datetime.fromisoformat(t).strftime('%H:%M') for t in raw_hourly['time']]
    
    data_rows = [dict(zip(raw_hourly.keys(), values)) for values in zip(*raw_hourly.values())]

    # 4. Crear el DataFrame con esquema aplicado
    df_valid = spark.createDataFrame(data_rows, schema=weather_struct)
    all_rows = [row.asDict() for row in df_valid.collect()]

    return all_rows

# --- Procesado de imágenes sin cambios ---
def process_accommodation_images(photo_urls: list, fs_client, city: str) -> None:
    for url in photo_urls:
        sha = string_to_sha256(url)
        path = f"landing_zone/accommodation_images/{city}/{sha}.jpg"
        trusted_path = f"trusted_zone/accommodation_images/{city}/{sha}.jpg"
        if file_exists(fs_client, path) and file_exists(fs_client, trusted_path):
            continue
        if file_exists(fs_client, path):
            data = fs_client.get_file_client(path).download_file().readall()
        else:
            res = req.get(url, stream=True)
            try:
                res.raise_for_status()
            except:
                continue
            data = res.content
            upload_file(fs_client, path, data)
        compressed = compress_image(data, max_width=800, quality=70)
        upload_file(fs_client, trusted_path, compressed)

BACKFILL_LOG = "/exploitation_zone/backfilled_files.txt"

def _load_backfill_log(hdfs_client: InsecureClient):
    if hdfs_client.status(BACKFILL_LOG, strict=False):
        with hdfs_client.read(BACKFILL_LOG) as reader:
            return reader.read().decode('utf-8').splitlines()
    return []

def _append_to_backfill_log(hdfs_client: InsecureClient, log_entry: str):
    entries = _load_backfill_log(hdfs_client)
    if log_entry not in entries:
        entries.append(log_entry)
        hdfs_client.write(BACKFILL_LOG, "\n".join(entries).encode('utf-8'), overwrite=True)

def _load_parquet(spark: SparkSession, hdfs_client: InsecureClient, data_type: str, city: str, filename: str):
    path = f"/exploitation_zone/{data_type}/{city}/{filename}"
    try:
        return spark.read.parquet(path)
    except Exception:
        return None

def _write_parquet_to_hdfs(df, hdfs_client: InsecureClient, data_type: str, city: str, filename: str):
    local_temp = f"/tmp/{filename}"
    df.write.mode('overwrite').parquet(local_temp)
    hdfs_target = f"/exploitation_zone/{data_type}/{city}/{filename}"
    hdfs_client.delete(hdfs_target, recursive=True)
    hdfs_client.upload(hdfs_target, local_temp, overwrite=True)
    log_entry = f"{data_type}/{city}/{filename}"
    _append_to_backfill_log(hdfs_client, log_entry)

def get_and_sync_accommodation(
    fs,
    start: datetime,
    end: datetime,
    cities: dict,
    query_template: dict,
    schema_file: str
):
    schema = load_json_schema(schema_file)
    delta = timedelta(days=1)

    for single_date in tqdm([start + i * delta for i in range((end - start).days + 1)]):
        arrival = single_date.strftime('%Y-%m-%d')
        departure = (single_date + delta).strftime('%Y-%m-%d')
        for city, dest_id in cities.items():
            landing_path = f"landing_zone/accommodation/{city}/{arrival}_{departure}.json"
            trusted_path = f"trusted_zone/accommodation/{city}/{arrival}_{departure}.json"

            # filtro: si existe en landing y trusted, nada que hacer
            if file_exists(fs, landing_path) and file_exists(fs, trusted_path):
                continue

            # obtener JSON: de landing o API
            if file_exists(fs, landing_path):
                raw = fs.get_file_client(landing_path).download_file().readall().decode('utf-8')
                data = json.loads(raw)
            else:
                params = dict(query_template, dest_id=dest_id, arrival_date=arrival, departure_date=departure)
                res = req.get(accommodation_endpoint, params=params)
                res.raise_for_status()
                data = res.json()
                upload_file(fs, landing_path, json.dumps(data).encode('utf-8'))

            # procesar imágenes
            photo_urls = [u for h in data['data']['hotels'] for u in h['property']['photoUrls']]
            process_accommodation_images(photo_urls, fs, city)

            # transformar y guardar registros en zona confiable
            docs = [process_accommodation_record(r, schema) for r in data['data']['hotels']]
            upload_file(fs, trusted_path, json.dumps(docs).encode('utf-8'))


def get_and_sync_weather(
    fs,
    start: datetime,
    end: datetime,
    coords: dict,
    query_template: dict,
    schema_file: str
):
    schema = load_json_schema(schema_file)
    delta = timedelta(days=1)

    for single_date in tqdm([start + i * delta for i in range((end - start).days + 1)]):
        prev_start = (single_date - timedelta(days=365)).strftime('%Y-%m-%d')
        for city, coord in coords.items():
            landing_path = f"landing_zone/weather/{city}/{prev_start}.json"
            trusted_path = f"trusted_zone/landing_zone/weather/{city}/{prev_start}.json"

            # filtro: si existe en landing y trusted, nada que hacer
            if file_exists(fs, landing_path) and file_exists(fs, trusted_path):
                continue

            # obtener JSON: de landing o API
            if file_exists(fs, landing_path):
                raw = fs.get_file_client(landing_path).download_file().readall().decode('utf-8')
                data = json.loads(raw)
            else:
                params = dict(query_template, latitude=coord['latitude'], longitude=coord['longitude'], start_date=prev_start, end_date=prev_start)
                res = req.get(weather_endpoint, params=params)
                res.raise_for_status()
                data = res.json()
                upload_file(fs, landing_path, json.dumps(data).encode('utf-8'))

            # transformar y guardar en zona confiable
            doc = process_weather_record(data, schema)
            upload_file(fs, trusted_path, json.dumps(doc).encode('utf-8'))

def list_files(fs_client, prefix: str) -> list:
    return [p.name for p in fs_client.get_paths(path=prefix) if not p.is_directory]


def find_missing_blobs(
    fs,
    landing_prefix: str
) -> list:
    missing = []
    for path in list_files(fs, 'landing_zone/' + landing_prefix):
        trusted_path = f"trusted_zone/{path.replace('landing_zone/', '', 1)}"
        if not file_exists(fs, trusted_path):
            missing.append(path)
    return missing


def backfill_missing(
    fs,
    landing_prefix: str,
    schema_file: str = None
) -> None:
    schema = load_json_schema(schema_file) if schema_file else None
    to_fill = find_missing_blobs(fs, landing_prefix)

    for landing_path in tqdm(to_fill):
        trusted_path = f"trusted_zone/{landing_path.replace('landing_zone/', '', 1)}"
        data_bytes = fs.get_file_client(landing_path).download_file().readall()

        # imágenes de alojamiento
        if landing_path.startswith('landing_zone/accommodation_images/'):
            compressed = compress_image(data_bytes, max_width=800, quality=70)
            upload_file(fs, trusted_path, compressed)
            continue

        # JSON de alojamiento
        if landing_path.startswith('landing_zone/accommodation/'):
            data = json.loads(data_bytes.decode('utf-8'))
            docs = [process_accommodation_record(r, schema) for r in data['data']['hotels']]
            upload_file(fs, trusted_path, json.dumps(docs).encode('utf-8'))
            continue

        # JSON de clima
        if landing_path.startswith('landing_zone/weather/'):
            data = json.loads(data_bytes.decode('utf-8'))
            doc = process_weather_record(data, schema)
            upload_file(fs, trusted_path, json.dumps(doc).encode('utf-8'))

def backfill_exploitation_zone(
    dl_client,
    hdfs_client: InsecureClient,
    spark: SparkSession,
    data_type: str
):
    log_entries = _load_backfill_log(hdfs_client)
    paths = list_files(dl_client, f"trusted_zone/")
    for path in tqdm(paths):
        parts = path.split('/')
        d_type, city, fname = parts[-3], parts[-2], parts[-1]
        full_path = f"trusted_zone/{data_type}/{city}/{fname}"

        log_key = f"{data_type}/{city}/{fname}"
        if log_key in log_entries:
            continue

        if data_type == 'accommodation' and d_type == data_type:
            data_bytes = dl_client.get_file_client(full_path).download_file().readall()
            docs = json.loads(data_bytes.decode('utf-8'))
            df = spark.read.json(spark.sparkContext.parallelize([json.dumps(d) for d in docs]))
            existing = _load_parquet(spark, hdfs_client, 'accommodation', city, 'AccommodationData.parquet')
            combined = df if existing is None else existing.unionByName(df, allowMissingColumns=True)
            _write_parquet_to_hdfs(combined, hdfs_client, 'accommodation', city, 'AccommodationData.parquet')

        elif data_type == 'weather' and d_type == data_type:
            data_bytes = dl_client.get_file_client(full_path).download_file().readall()
            raw = json.loads(data_bytes.decode('utf-8'))
            df = spark.read.json(spark.sparkContext.parallelize([json.dumps(raw)]))
            existing = _load_parquet(spark, hdfs_client, 'weather', city, 'WeatherData.parquet')
            combined = df if existing is None else existing.unionByName(df, allowMissingColumns=True)
            _write_parquet_to_hdfs(combined, hdfs_client, 'weather', city, 'WeatherData.parquet')

        elif data_type == 'accommodation_images' and d_type == data_type:
            data_bytes = dl_client.get_file_client(full_path).download_file().readall()
            hdfs_target = f"/exploitation_zone/accommodation_images/{city}/{fname}"
            hdfs_client.write(hdfs_target, data_bytes, overwrite=True)
            _append_to_backfill_log(hdfs_client, f"accommodation_images/{city}/{fname}")

In [ ]:
from contextlib import closing
from datetime import datetime, timedelta
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from hdfs import InsecureClient
import os


def get_and_sync_accommodation(
    fs,
    start: datetime,
    end: datetime,
    cities: dict,
    query_template: dict,
    schema_file: str,
    spark: SparkSession,
    hdfs_client: InsecureClient
):
    schema = json.load(open(schema_file))
    delta = timedelta(days=1)

    for single_date in [start + i * delta for i in range((end - start).days + 1)]:
        arrival = single_date.strftime('%Y-%m-%d')
        departure = (single_date + delta).strftime('%Y-%m-%d')
        for city, dest_id in cities.items():
            # read trusted docs
            trusted_path = f"trusted_zone/accommodation/{city}/{arrival}_{departure}.json"
            if not file_exists(fs, trusted_path):
                continue
            data_bytes = fs.get_file_client(trusted_path).download_file().readall()
            docs = json.loads(data_bytes.decode('utf-8'))
            # create spark DF from docs
            exploitation_df = spark.read.json(spark.sparkContext.parallelize([json.dumps(d) for d in docs]))
            # add date columns
            exploitation_df = exploitation_df.withColumn('check_in_date', lit(arrival)) \
                                               .withColumn('check_out_date', lit(departure))
            # read existing parquet
            existing_df = _load_parquet(spark, hdfs_client, 'accommodation_data', city, 'AccommodationData.parquet')
            if existing_df is None:
                combined = exploitation_df
            else:
                combined = existing_df.unionByName(exploitation_df, allowMissingColumns=True)
            # write back
            _write_parquet_to_hdfs(combined, hdfs_client, 'accommodation_data', city, 'AccommodationData.parquet')


def get_and_sync_weather(
    fs,
    start: datetime,
    end: datetime,
    coords: dict,
    query_template: dict,
    schema_file: str,
    spark: SparkSession,
    hdfs_client: InsecureClient
):
    delta = timedelta(days=1)

    for single_date in [start + i * delta for i in range((end - start).days + 1)]:
        date_str = single_date.strftime('%Y-%m-%d')
        for city in coords.keys():
            trusted_path = f"trusted_zone/weather/{city}/{date_str}.json"
            if not file_exists(fs, trusted_path):
                continue
            data_bytes = fs.get_file_client(trusted_path).download_file().readall()
            raw = json.loads(data_bytes.decode('utf-8'))
            # process record
            rec = process_weather_record(raw, json.load(open(schema_file)))
            # create spark DF
            exploitation_df = spark.read.json(spark.sparkContext.parallelize([json.dumps(rec)]))
            # add date column
            exploitation_df = exploitation_df.withColumn('Date', lit(date_str))
            # read existing parquet
            existing_df = _load_parquet(spark, hdfs_client, 'weather_data', city, 'WeatherData.parquet')
            if existing_df is None:
                combined = exploitation_df
            else:
                combined = existing_df.unionByName(exploitation_df, allowMissingColumns=True)
            _write_parquet_to_hdfs(combined, hdfs_client, 'weather_data', city, 'WeatherData.parquet')



# Pipeline

In [19]:
destination_ids = {
    "Barcelona": "-372490",
    "Rome": "-126693",
    "Madrid": "-390625",
    "Paris": "-1456928"
}

destination_coords = {
    'Barcelona': {'latitude': 41.3874, 'longitude': 2.1686},
    'Paris': {'latitude': 48.8575, 'longitude': 2.3514},
    'Madrid': {'latitude': 40.4167, 'longitude': 3.7033},
    'Rome': {'latitude': 41.8967, 'longitude': 12.4822}
}

accommodation_endpoint = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"
weather_endpoint = 'https://archive-api.open-meteo.com/v1/archive'

headers = {
    "x-rapidapi-key": os.environ["RAPID_API_KEY"],
    "x-rapidapi-host": os.environ["RAPID_API_HOST"]
}

accommodation_query = {
    "dest_id": '',
    "search_type": "CITY",
    "arrival_date": '',
    "departure_date": '',
    "adults": "2",
    "children_age": "0",
    "room_qty": "1",
    "page_number": "1",
    "units": "metric",
    "temperature_unit": "c",
    "languagecode": "en-us",
    "currency_code": "EUR"
}

weather_metrics = 'temperature_2m,rain,snowfall,precipitation,cloud_cover,wind_speed_10m,sunshine_duration'

weather_query = {
    'latitude': '',
    'longitude': '',
    'hourly': weather_metrics,
    'start_date': '',
    'end_date': ''
}

In [7]:
# Initialize DataLakeServiceClient once
datalake_client = DataLakeServiceClient.from_connection_string(connection_string)
file_system_client = datalake_client.get_file_system_client(storage_container_name)
hdfs_service = HdfsClient(os.environ['HDFS_URL'])

In [24]:
# parameters
start = datetime.strptime('2025-07-01', '%Y-%m-%d')
end = datetime.strptime('2025-07-02', '%Y-%m-%d')
cities = list(destination_ids.keys())

# sync on the fly
get_and_sync_accommodation(file_system_client, start, end, destination_ids, accommodation_query, 'accommodation_schema.json')
get_and_sync_weather(file_system_client, start, end, destination_coords, weather_query, 'weather_schema.json')

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]


HTTPError: 429 Client Error: Too Many Requests for url: https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels?dest_id=-372490&search_type=CITY&arrival_date=2025-07-01&departure_date=2025-07-02&adults=2&children_age=0&room_qty=1&page_number=1&units=metric&temperature_unit=c&languagecode=en-us&currency_code=EUR

In [ ]:
backfill_missing(file_system_client, 'accommodation/', 'accommodation_schema.json')
backfill_missing(file_system_client, 'accommodation_images/')
backfill_missing(file_system_client, 'weather/', 'weather_schema.json')

100%|██████████| 377/377 [02:11<00:00,  2.87it/s]


In [90]:
backfill_exploitation_zone(file_system_client, hdfs_service, spark, 'accommodation')
backfill_exploitation_zone(file_system_client, hdfs_service, spark, 'accommodation_images')
backfill_exploitation_zone(file_system_client, hdfs_service, spark, 'weather')

100%|██████████| 2663/2663 [08:05<00:00,  5.48it/s]  


In [11]:
def show_exploitation_parquet(spark: SparkSession, data_type: str, city: str, filename: str = "*.parquet", limit: int = 20):
    path = f"hdfs://localhost:9000/exploitation_zone/{data_type}/{city}/{filename}"
    try:
        df = spark.read.parquet(path)
        df.show(limit, truncate=False)
        return df
    except Exception as e:
        print(f"Error reading parquet from {path}: {e}")
        return None

In [95]:
show_exploitation_parquet(spark, 'accommodation', 'Madrid', 'AccommodationData.parquet', 25)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------------------+-------------------------------------+--------------------+-------------------------+--------------------------+---------------------+--------------------------+---------------------------+--------------------+-----------------+-----------+--------------------+--------------------+------------------------+-----------------+------------------+--------------------+---------------------------------------------------+---------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-----

DataFrame[accessibilityLabel: string, hotel_id: bigint, property_accuratePropertyClass: bigint, property_blockIds: array<string>, property_checkinDate: string, property_checkin_fromTime: string, property_checkin_untilTime: string, property_checkoutDate: string, property_checkout_fromTime: string, property_checkout_untilTime: string, property_countryCode: string, property_currency: string, property_id: bigint, property_isFirstPage: boolean, property_isPreferred: boolean, property_isPreferredPlus: boolean, property_latitude: double, property_longitude: double, property_mainPhotoId: bigint, property_name: string, property_optOutFromGalleryChanges: bigint, property_photoHash: array<string>, property_position: bigint, property_priceBreakdown_benefitBadges: array<array<string>>, property_priceBreakdown_excludedPrice_currency: string, property_priceBreakdown_excludedPrice_value: double, property_priceBreakdown_grossPrice_currency: string, property_priceBreakdown_grossPrice_value: double, prop

In [96]:
show_exploitation_parquet(spark, 'weather', 'Madrid', 'WeatherData.parquet', 25)

+-----------+-------------+----+--------+-----------------+--------------+-----+----------------+--------------+
|cloud_cover|precipitation|rain|snowfall|sunshine_duration|temperature_2m|time |timestamp       |wind_speed_10m|
+-----------+-------------+----+--------+-----------------+--------------+-----+----------------+--------------+
|0          |0.0          |0.0 |0.0     |0.0              |23.0          |00:00|2024-06-28T00:00|14.8          |
|9          |0.0          |0.0 |0.0     |0.0              |23.3          |01:00|2024-06-28T01:00|18.4          |
|1          |0.0          |0.0 |0.0     |0.0              |23.0          |02:00|2024-06-28T02:00|16.0          |
|5          |0.0          |0.0 |0.0     |0.0              |22.9          |03:00|2024-06-28T03:00|16.4          |
|65         |0.0          |0.0 |0.0     |0.0              |22.9          |04:00|2024-06-28T04:00|17.0          |
|31         |0.0          |0.0 |0.0     |0.0              |22.9          |05:00|2024-06-28T05:00

DataFrame[cloud_cover: bigint, precipitation: double, rain: double, snowfall: double, sunshine_duration: double, temperature_2m: double, time: string, timestamp: string, wind_speed_10m: double]